# Prompting LLMs for Patient-Oriented Language 

In [1]:
import os
import transformers
import json

import torch
import tqdm

from typing import Dict, List, Optional

from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer
from datasets import load_dataset


## Load dataset

In [2]:
data = load_dataset("csv", data_files="pleaseee.csv")

Found cached dataset csv (/linkhome/rech/genlig01/ulj12fo/.cache/huggingface/datasets/csv/default-c10b8bc08219fdf5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|##########| 1/1 [00:00<00:00,  4.05it/s]


In [3]:
#data = load_dataset("Mathilde/test_data_pol")
subset = data['train']  # TODO changer en test dans le ds original
print(subset)
"""
statements_pol = data['train']['statement_pol']
statements_med = data['train']['statement_med']
premises =  data['train']['premise']
"""

Dataset({
    features: ['id', 'topic_id', 'statement_medical', 'statement_pol', 'premise', 'NCT_title', 'NCT_id', 'label'],
    num_rows: 1578
})


"\nstatements_pol = data['train']['statement_pol']\nstatements_med = data['train']['statement_med']\npremises =  data['train']['premise']\n"

## Load Model

TODO: change the model's path

In [4]:
model_path = "../../baselines/Qwen2.5-14B-Instruct" # "/gpfsdswork/dataset/HuggingFace_Models/meta-llama/Llama-2-7b-chat-hf" #"/lustre/fsn1/projects/rech/hjp/ulj12fo/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

In [5]:
streamer = TextStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=True, device_map="auto",)  # torch_dtype=torch.float16, low_cpu_mem_usage=True

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|##########| 8/8 [07:48<00:00, 58.57s/it]


## Format the prompt

In [7]:
def get_input_text(premise, hypothesis):
    # TODO the persona option + the wrapping of the pol + med "patients with this medical profile..."
    options_prefix = "Answer only with: \n- " #"OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment","Contradiction"])
    return f"{premise} \n Question: Does the previous eligibility criteria imply that the following patient can participate to the trial?\n {hypothesis}\n {options_}"
    #return f"{premise} \n Question: Does this imply that {hypothesis}? {options_}"
    # Does the previous eligibility criteria imply that the following patient can participate to the trial?
    # return f"Classification: {premise} \n Question: Does this imply that {hypothesis}? Entailment or Contradiction?Answer:"

In [8]:
# subset[149]

In [21]:
samples = []
# TODO adapt in function of the med or POL (l.5)
for instance in subset:
    if instance['topic_id'] == 30:
        premise = instance['premise']
        sentence = f"Eligibility criteria of the trial are:\n {premise}"
        input_text = get_input_text(sentence, instance['statement_medical'])
        # temp = {"text":input_text, "label":sample['label']}
        temp = {"text":input_text, "label":0}
        print(input_text)
        samples.append(temp)

Eligibility criteria of the trial are:
 Inclusion Criteria:

          1. Meet the Clinical Classification Criteria for Osteoarthritis of the Knee, diagnosed by
             specialist.

          2. Aged 40-75 years old;

          3. History of chronic knee pain for more than 3 months, and complain of knee pain on most
             days of the past month;

          4. Kellgren and Lawrence Grades 1-3;

          5. With related knee pain in one or both knees rated > 4 cm on a 10 cm Visual Analog
             Scale(VAS) (VAS:0 to 10)

          6. Who volunteer to participate and sign the consent form .

        Exclusion Criteria:

          1. Knee osteoarthritis patients with gout, infection, tumor, autoimmune diseases, trauma
             or other causes of knee pain or knee deformities;

          2. Patients who have local skin damage, poor skin conditions or coagulant dysfunction and
             patients who are not suitable for acupuncture;

          3. Patients who are acc

## Define the chat function

TODO: adjust the hyperparameters

In [22]:
def chat(
    query: str,
    history: Optional[List[Dict]] = None,
    temperature: float = 0.7,
    top_p: float = 1.0,
    top_k: float = 0,
    repetition_penalty: float = 1.1,
    max_new_tokens: int = 1024, # 1024,
    **kwargs,
):
    if history is None:
        history = []

    history.append({"role": "user", "content": query})

    input_ids = tokenizer.apply_chat_template(history, add_generation_prompt=True, return_tensors="pt").to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            **kwargs,
        ),
        streamer=streamer,
        return_dict_in_generate=True,
    )

    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    history.append({"role": "assistant", "content": generated_text})

    return generated_text, history

In [23]:
samples[0]['text']

"Eligibility criteria of the trial are:\n Inclusion Criteria:\n\n          1. Meet the Clinical Classification Criteria for Osteoarthritis of the Knee, diagnosed by\n             specialist.\n\n          2. Aged 40-75 years old;\n\n          3. History of chronic knee pain for more than 3 months, and complain of knee pain on most\n             days of the past month;\n\n          4. Kellgren and Lawrence Grades 1-3;\n\n          5. With related knee pain in one or both knees rated > 4 cm on a 10 cm Visual Analog\n             Scale(VAS) (VAS:0 to 10)\n\n          6. Who volunteer to participate and sign the consent form .\n\n        Exclusion Criteria:\n\n          1. Knee osteoarthritis patients with gout, infection, tumor, autoimmune diseases, trauma\n             or other causes of knee pain or knee deformities;\n\n          2. Patients who have local skin damage, poor skin conditions or coagulant dysfunction and\n             patients who are not suitable for acupuncture;\n\n      

In [ ]:
response, history = chat(samples[0]['text'], history=None)  # 21: 2882 // 30: 

Contradiction

The patient's presentation includes a tender swelling in the popliteal fossa and calf along with an enlarged effusion due to bending and straightening the knee, which may indicate other causes of her knee pain beyond osteoarthritis, such as possible vascular issues (e.g., deep vein thrombosis). This could fall under the exclusion criteria point number 1 ("Knee osteoarthritis patients with ... other causes of knee pain..."). Therefore, based on this information, it contradicts the full eligibility to participate merely under the inclusion criteria for osteoarthritis without further clarification or exclusion of other potential 

In [12]:
#samples[52]['text']

## Call the chat function on the whole test set

In [ ]:
labels = []
pred = []
#with torch.inference_mode():
for sample in tqdm.tqdm(samples):
    labels.append(sample["label"])
    # input_ids = tokenizer(sample["text"], return_tensors="pt",).input_ids.to("cuda")
    # input_ids = tokenizer.apply_chat_template(sample["text"], return_tensors="pt",).to("cuda")
    # outputs = model.generate(input_ids, max_new_tokens=20)
    response, history = chat(sample['text'], history=None)
    pred.append(response)

  0%|          | 0/1578 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


- Contradiction




  0%|          | 1/1578 [00:06<2:59:36,  6.83s/it]

Entailment

The


  0%|          | 2/1578 [00:08<1:40:15,  3.82s/it]

...

 76%|#######5  | 1195/1578 [24:52<07:54,  1.24s/it]

Entailment

The


 76%|#######5  | 1196/1578 [24:53<07:47,  1.22s/it]

Entailment

The


 76%|#######5  | 1197/1578 [24:54<07:42,  1.21s/it]

In [ ]:
pred

In [ ]:
# pred = [p[5:][:-4].strip() for p in pred]

## Parse model's output

TODO: adapt the pattern in function of the model's output style

In [ ]:
pattern = "entailment|contradiction|Entailment|Contradiction|entail|Neutral|neutral"

In [ ]:
import re

# Function to match text with regular expression
def match_text_with_regexp(text, pattern):
    text = text.strip()
    # print(text)
    # Compile the regular expression pattern
    regexp = re.compile(pattern)
    
    # Search for a match in the text
    match = regexp.search(text)
    # print(match)
    
    if match:
        # If a match is found, return the matched text
        return match.group()
    else:
        # If no match is found
        return None

parsed_preds = []

for p in pred:
    # text = pred[0]
    pattern = "entailment|contradiction|Entailment|Contradiction|entail"
    
    result = match_text_with_regexp(p, pattern)
    
    if result:
        if result in ['entailment', 'entail', 'yes', 'Yes']:
            result = 'Entailment'
        elif result in ['contradiction', 'contradicts', 'no', 'No']:
            result = 'Contradiction'
        #elif result in ['neutral', 'Neutral']:
            #result = 'Neutral'
        # print(f"Match found: {result}")
        parsed_preds.append(result) 
    else:
        # print("No match found.")
        # if nothing exploitable predicted --> assert "Neutral"
        parsed_preds.append("Contradiction")
        


In [ ]:
parsed_preds

In [ ]:
set(parsed_preds)
from collections import Counter
Counter(parsed_preds)

## Save the predictions in a JSON file

In [ ]:
prediction_dict = {}
for _id,pred_x in zip(data['train']['id'], parsed_preds):
    prediction_dict[str(_id)] = {"Prediction":pred_x}

In [ ]:
# from sklearn.metrics import f1_score
# uuid_list = list(prediction_dict.keys())
# results_pred = []
# gold_labels = []
# for i in range(len(uuid_list)):
#     if prediction_dict[uuid_list[i]]["Prediction"] in ["Entailment", "Yes"]:
#         results_pred.append(1)
#     else:
#         results_pred.append(0)
#     if data[uuid_list[i]]["Label"] in ["Entailment", "No"]:
#         gold_labels.append(1)
#     else:
#         gold_labels.append(0)
# f1_score(gold_labels,results_pred)

In [ ]:
json.dump(prediction_dict, open("results_qwen_2-5_14B_zs_pol.json",'w'),indent=4)

In [ ]:
!zip results_llama_zs.zip results.json

In [ ]:
prediction_dict